In [1]:
# from pathlib import Path, PureWindowsPath
# import re
# import itertools
# import openpyxl

In [2]:
from pathlib import Path
import os
from pdf2image import convert_from_path

import cv2
import pytesseract as pt
from PIL import Image
import pandas as pd

In [3]:
### ca. 190 min!!

### Insert full path of input directory containing the election programs as .pdf file 
input_dir = Path("C:/DHBW WWI DS(A) Studium/S5/Natural Language Processing/NLP_Projekt/input/")

In [4]:
def pdf_2_images(input_dir=input_dir):
    pdf_counter = 1
    ### Iterate through pdf input directory and process only PDF files via .pdf extension
    for file in os.listdir(input_dir):
        if file.endswith(".pdf"):
            print(f"Processing PDF file {pdf_counter}: {file}")
            pdf_counter += 1

            ### Create entire filepath for each PDF file
            pdf_file_path = input_dir/file
            fn = file.split(".")[0]

            ### Create a sub directory inside the "input" directory for each PDF
            #############################################
            sub_dir_path = input_dir/fn
            ### Check if sub_dir already exists, if not --> create sub_dir for each PDF
            if not os.path.exists(sub_dir_path):
                os.mkdir(os.path.join(sub_dir_path))
                print(f"New SUB_DIR created: {sub_dir_path}")
            else:
                print(f"SUB_DIR already exists: {sub_dir_path}")

            ### Generate image file in sub_dir for each page in PDF file
            #############################################
            pages = convert_from_path(pdf_file_path, dpi=500)
            for page_index, page in enumerate(pages):
                ### Name schema of the image filenames
                img_fn = f"{fn}_Page_{str(page_index)}"+".png"
                ### Check if image file already exists
                if not os.path.exists(sub_dir_path/img_fn):
                    print(f"Saving {fn} PDF page {page_index}")
                    page.save(sub_dir_path/img_fn, "PNG")
                    continue
                else:
                    print("File already exists --> SKIP")
                    continue
    print("PROCESSING FINISHED!")

pdf_2_images(input_dir=input_dir)

Processing PDF file 1: AFD_Wahlprogramm_2021.pdf
New SUB_DIR created: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021
Saving AFD_Wahlprogramm_2021 PDF page 0
Saving AFD_Wahlprogramm_2021 PDF page 1
Saving AFD_Wahlprogramm_2021 PDF page 2
Saving AFD_Wahlprogramm_2021 PDF page 3
Saving AFD_Wahlprogramm_2021 PDF page 4
Saving AFD_Wahlprogramm_2021 PDF page 5
Saving AFD_Wahlprogramm_2021 PDF page 6
Saving AFD_Wahlprogramm_2021 PDF page 7
Saving AFD_Wahlprogramm_2021 PDF page 8
Saving AFD_Wahlprogramm_2021 PDF page 9
Saving AFD_Wahlprogramm_2021 PDF page 10
Saving AFD_Wahlprogramm_2021 PDF page 11
Saving AFD_Wahlprogramm_2021 PDF page 12
Saving AFD_Wahlprogramm_2021 PDF page 13
Saving AFD_Wahlprogramm_2021 PDF page 14
Saving AFD_Wahlprogramm_2021 PDF page 15
Saving AFD_Wahlprogramm_2021 PDF page 16
Saving AFD_Wahlprogramm_2021 PDF page 17
Saving AFD_Wahlprogramm_2021 PDF page 18
Saving AFD_Wahlprogramm_2021 PDF page 19
Saving AFD_Wahlprogramm

In [5]:
# ### Raw string extraction as .txt file of the image files combined per PDF/sub_dir
# def get_txt_from_images(input_dir=input_dir):
#     ### Create a list of the sub directories for later iteration
#     sub_dir_list = []
#     for dir in os.listdir(input_dir):
#         if (".pdf" not in dir) & (".txt" not in dir) & (".xlsx" not in dir):
#             sub_dir_list.append(dir)
#         else:
#             continue
#     print(f"List of Sub_DIRs: \n{sub_dir_list}")

#     ### Iterate over the list of sub directories and generate the sub_dir path
#     for sub_dir in sub_dir_list:
#         sub_dir_path = input_dir/sub_dir

#         ### Iterating through folder would lead to incorrect order 1, 10, 2, 3 --> Therefore iterate based on the filename numbering
#         ### Generate filename_structure by removing the number of the page (REMOVE --> _0.png)
#         fn_structure = os.listdir(sub_dir_path)[0]
#         fn_structure = fn_structure.split("_")[0:-1]
#         fn_structure = "_".join(fn_structure)

#         ### .txt filepath
#         txt_file = Path(input_dir/f"{sub_dir}.txt")
#         with open(txt_file, "a", encoding="utf-8") as output_file:

#             ### Generate the correct order for extraction 
#             ### Iterating through folder would lead to incorrect order 1, 10, 2, 3 --> Therefore iterate based on the filename numbering
#             for img_no in range(0 , len(os.listdir(sub_dir_path))):
#                 ### use the fn_structure for each party and generate filenames via iterating image numbers
#                 img_file = fn_structure+f"_{img_no}.png"
#                 print(f"Extracting text from image: {img_file}")

#                 ## Recognize the text as string in image using pytesserct
#                 ### German language package used "deu"
#                 text = pt.image_to_string(
#                 Image.open(sub_dir_path/img_file), 
#                 lang="deu", 
#                 )
#                 output_file.write(text)
#             print(f"Save text into .txt file: {txt_file}")

#     print("Text Extraction FINISHED!")

# get_txt_from_images(input_dir=input_dir)

In [6]:
### Dataframe extraction as .csv file of the image files combined per PDF/sub_dir
def get_df_from_images(input_dir=input_dir):

    sub_dir_list = []
    for dir in os.listdir(input_dir):
        if (".pdf" not in dir) & (".txt" not in dir) & (".xlsx" not in dir):
            sub_dir_list.append(dir)
        else:
            continue
    print(f"List of Sub_DIRs: \n{sub_dir_list}")

    ### Initiate MASTER_DF with all parties
    master_df = pd.DataFrame()

    ### Iterate over the list of sub directories and generate the sub_dir path
    for sub_dir in sub_dir_list:
        sub_dir_path = input_dir/sub_dir

        ### Iterating through folder would lead to incorrect order 1, 10, 2, 3 --> Therefore iterate based on the filename numbering
        ### Generate filename_structure by removing the number of the page (REMOVE --> _0.png)
        fn_structure = os.listdir(sub_dir_path)[0]
        fn_structure = fn_structure.split("_")[0:-1]
        fn_structure = "_".join(fn_structure)

        ### Initiate an empty DF for each sub_dir
        comb_df = pd.DataFrame()
        ### Generate the correct order for extraction 
        ### Iterating through folder would lead to incorrect order 1, 10, 2, 3 --> Therefore iterate based on the filename numbering
        for img_no in range(0 , len(os.listdir(sub_dir_path))):
            ### use the fn_structure for each party and generate filenames via iterating image numbers
            img_fname = fn_structure+f"_{img_no}.png"
            img_fpath = sub_dir_path/img_fname
            print(f"Extracting DF from image: {img_fpath}")

            ### Read images and create pytesseract DF with coordinates for ecach image and combine for each sub_dir/PDF
            img_file = cv2.imread(str(img_fpath), 1)
            sub_df   = pt.image_to_data(
                            ### German language package used "deu"
                            img_file, lang="deu",
                            ### Pytesseract page segmentation mode (--psm 6) -> assume a single uniform block of text. 
                            ### https://ai-facets.org/tesseract-ocr-best-practices/
                            config= r'--oem 1 --psm 6' ,
                            output_type = pt.Output.DATAFRAME
                            )
            ### REMOVE the pytesseract page_num (always 1)
            sub_df = sub_df.drop(columns=["page_num"], axis=1)
            ### ADD the correct page_num into the column for each sub_DF
            sub_df["page_num"] = [img_no for i in range(0, len(sub_df))] # image no which counts based on the length of the sub_dir --> MAYBE Change...
            comb_df = pd.concat([comb_df, sub_df])

        ### ADD the sub_dir name 
        comb_df["party"] = [sub_dir for i in range(0, len(comb_df))]
        ### Save separated DF for each sub_dir
        comb_df.to_excel(Path(input_dir/f"{sub_dir}.xlsx"))
        print(f"Save DF into .xlsx file:")

        ### CONCATENATE the combined party_df into one MASTER_DF
        master_df = pd.concat([master_df, comb_df])

    ### SAVE and return the full Master_DF with all parties
    master_df.to_excel(Path(input_dir/f"MASTER.xlsx"))
    print("PROCESSING FINISHED!")
    return(master_df)
    
get_df_from_images(input_dir=input_dir)

List of Sub_DIRs: 
['AFD_Wahlprogramm_2021', 'CDU-CSU_Wahlrprogramm_2021', 'DIE_GRUENEN_Wahlprogramm_2021', 'DIE_LINKE_Wahlprogramm_2021', 'FDP_Wahlprogramm_2021', 'SPD_Wahlprogramm_2021']
Extracting DF from image: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021\AFD_Wahlprogramm_2021_Page_0.png
Extracting DF from image: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021\AFD_Wahlprogramm_2021_Page_1.png
Extracting DF from image: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021\AFD_Wahlprogramm_2021_Page_2.png
Extracting DF from image: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021\AFD_Wahlprogramm_2021_Page_3.png
Extracting DF from image: C:\DHBW WWI DS(A) Studium\S5\Natural Language Processing\NLP_Projekt\input\AFD_Wahlprogramm_2021\AFD_Wahlprogramm_2021_Page_4.png
Extracting DF from image: C:\DH

,level,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,page_num,party
0,1,0,0,0,0,0,0,2973,4252,-1.000000,NaN,0,AFD_Wahlprogramm_2021
1,2,1,0,0,0,332,1669,2265,638,-1.000000,NaN,0,AFD_Wahlprogramm_2021
2,3,1,1,0,0,332,1669,2280,638,-1.000000,NaN,0,AFD_Wahlprogramm_2021
3,4,1,1,1,0,338,1669,2259,355,-1.000000,NaN,0,AFD_Wahlprogramm_2021
4,5,1,1,1,1,338,1717,1093,188,96.505013,Deutschland.,0,AFD_Wahlprogramm_2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,4,1,1,2,0,3409,5396,475,85,-1.000000,NaN,65,SPD_Wahlprogramm_2021
7,5,1,1,2,1,3409,5396,307,85,0.000000,ZT,65,SPD_Wahlprogramm_2021
8,5,1,1,2,2,3742,5396,142,85,0.000000,"aT,",65,SPD_Wahlprogramm_2021
9,4,1,1,3,0,3409,5504,235,85,-1.000000,NaN,65,SPD_Wahlprogramm_2021


########################DONE######################################

# Zahlen raus

In [7]:
# from matplotlib import pyplot as plt
# import cv2

# def display_bbx_img(df, fp):
#     for index, rows in df.iterrows():
#         (left, top, width, height) = df.at[index, 'left'], df.at[index, 'top'], df.at[index, 'width'], df.at[index, 'height']
#         cv2.rectangle(img_file, (left, top), (left + width, top + height), (255, 0, 0), 10)

#     bbx_img = plt.figure(figsize = (7, 10))
#     plt.imshow(img_file, aspect='auto')
#     plt.show()

#     return bbx_img

In [8]:
# from matplotlib import pyplot as plt
# import cv2

# nan_df = df[df["text"].isna()==True]

# # cv2.rectangle(img_file, (left, top), (left-5 + width+10, top-5 + height+10), (255, 0, 0), 10)

# for index, rows in nan_df.iterrows():
#     (left, top, width, height) = nan_df.at[index, 'left'], nan_df.at[index, 'top'], nan_df.at[index, 'width'], nan_df.at[index, 'height']
#     cv2.rectangle(img_file, (left, top), (left + width, top + height), (255, 0, 0), 10)


# plt.figure(figsize = (7, 10))
# plt.imshow(img_file, aspect='auto')
# plt.show()